# Intro

qbit上のシュレーディンガー方程式を解くためのライブラリを作るための、try/err

# Theoretical background

$\left|\psi\right> = c_i\left|s_1^i\cdots s_N^i\right> $

$H = J_{ij}\sigma^z_i\sigma^z_j - h_i\sigma_i^z$

# Import

In [15]:
import numpy as np

# Slow ising model

In [19]:
from itertools import product

Spin = int
Spins = List[Spin]

def all_spins(alpha: Spin, beta: Spin, num_spin: int) -> List[Spins]:
    sigmas_list = [[alpha, beta] for _ in range(num_spin)]
    return [list(sigmas) for sigmas in product(*sigmas_list)]


class IsingBasisSet:
    def __init__(self, sigmas_list):
        self.num_spin = len(sigmas_list[0])
        for sigmas in sigmas_list:
            assert(self.num_spin == len(sigmas))
        self.sigmas_list = sigmas_list

    def dim(self)->int:
        return len(self.sigmas_list)

    def single_flip(self, sigmas_i: list, sigmas_j: list) ->(bool, int):
        n = self.num_spin
        if n != len(sigmas_j):
            return False, -1
        elif np.dot(sigmas_i, sigmas_j) == n-2:
            i0s = [i for i in range(n) if sigmas_i[i] != sigmas_j[i]]
            return True, i0s[0]
        else:
            return False, -1


class IsingModelSlow:
    """ represent ising model Hamiltonian
        H = J_{ij}(\sigma^z_i)(\sigma^z_j) + (h_i)(\sigma^z_i)
    """
    def __init__(self, basis_set: IsingBasisSet, js: Dict[Tuple[int, int], float], hs: List[float]):
        super().__init__(basis_set.dim())
        self.basis_set = basis_set
        self.js = js
        self.hs = hs
        self.energies = None

    def calc_tid_energies(self) -> np.array:
        if self.energies is None:
            self.energies = np.array([
                -np.sum([j_ij*sigmas[i]*sigmas[j]
                         for ((i, j), j_ij) in self.js.items()])
                - np.dot(self.hs, sigmas)
                for sigmas in self.basis_set.sigmas_list])
        return self.energies

    def calc_tid_i_dot_c(self, c: np.ndarray) -> np.ndarray:
        return np.multiply(self.calc_tid_energies(), c)

    def to_str(self, n: int):
        return ("{0}IsingModel\n"
            "{0} num_spin: {1}\n"
            "{0} dim: {2}\n"
            "{0} js: {3}\n"
            "{0} hs: {4}\n").format(
            " "*n, self.basis_set.num_spin,
            self.basis_set.dim(), self.js, self.hs)

In [21]:
all_spins(0, 1, 4)

[[0, 0, 0, 0],
 [0, 0, 0, 1],
 [0, 0, 1, 0],
 [0, 0, 1, 1],
 [0, 1, 0, 0],
 [0, 1, 0, 1],
 [0, 1, 1, 0],
 [0, 1, 1, 1],
 [1, 0, 0, 0],
 [1, 0, 0, 1],
 [1, 0, 1, 0],
 [1, 0, 1, 1],
 [1, 1, 0, 0],
 [1, 1, 0, 1],
 [1, 1, 1, 0],
 [1, 1, 1, 1]]

# Type definition

In [1]:
from typing import List, Dict, Callable, Tuple

In [9]:
class QuantumState:
    """
    Quantum state on q-bits.
    
    self.state_vector[i] gives 
    """
    def __init__(self, qbit_count: int):
        self.qbit_count = qbit_count
        self.dim = 2**qbit_count
        self.state_vector = np.zero(self.qbit_count)
        self.state_vector[0] = 1.0
        
    def 

In [7]:
class IsingModel:
    def __init__(self, 
                 J_ij: Dict[Tuple[int, int], float], 
                 h_i: List[float], 
                 a_t: Callable[[float], float],
                 b_t: Callable[[float], float]):
        self.J_ij = J_ij
        self.h_i = h_i
        self.a_t = a_t
        self.b_t = b_t

# Traverse magnetic

In [11]:
class IsingTraverseMagnetModel(TIDQuantumModel):
    def __init__(self, basis_set: IsingBasisSet):
        super().__init__(basis_set.dim())
        self.basis_set = basis_set
        self.h = None

    def calc_tid_hamiltonian(self):
        sigmas_list = self.basis_set.sigmas_list
        dim = self.dim
        data = []
        row = []
        col = []
        for i in range(dim):
            for j in range(i):
                sigma_i = sigmas_list[i]
                sigma_j = sigmas_list[j]
                res = self.basis_set.single_flip(sigma_i, sigma_j)
                if res[0]:
                    data.append(-1.0)
                    data.append(-1.0)
                    row.append(i)
                    row.append(j)
                    col.append(j)
                    col.append(i)

        return bsr_matrix((data, (row, col)), shape=(dim, dim))

    def calc_tid_i_dot_c(self, c: np.ndarray) -> np.ndarray:
        if self.h is None:
            self.h = self.calc_tid_hamiltonian()
        return self.h*c

    def to_str(self, n: int):
        return ("{0}IsingTraverseMagnetModel\n"
                "{0} num_spin: {1}\n"
                "{0} dim: {2}\n").format(" "*n, self.basis_set.num_spin, self.basis_set.dim())

NameError: name 'TIDQuantumModel' is not defined